In [8]:
import onnx
import joblib
import xgboost
from onnxmltools.convert import convert_xgboost

from skl2onnx.common.data_types import FloatTensorType, DoubleTensorType


In [23]:
class DataSet:
    def __init__(self, name, ncols, TARGET_OPSET):
        self.name = name
        self.ncols = ncols
        self.TARGET_OPSET = TARGET_OPSET

datasets = [
    DataSet("Acute_Inflammations", 6, 15),
    DataSet("Breast_Cancer", 9, 15),
    DataSet("Chronic_Kidney_Disease", 24, 15),
    DataSet("Heart_Disease", 13, 15),
    DataSet("Heart_Failure_Clinical_Records", 12, 15),
    DataSet("Lymphography", 18, 15),
    DataSet("Parkinsons", 22, 15),
    ]


In [25]:
for dataset in datasets:
    # load the xgboost model
    model_path = f"{dataset.name}/{dataset.name}.pkl"
    xgboost_clf = joblib.load(model_path)
    onnx_model_converted = convert_xgboost(xgboost_clf, 'tree-based classifier',
                             [('input', FloatTensorType([1, dataset.ncols]))],
                             target_opset=dataset.TARGET_OPSET)
    onnx.save_model(onnx_model_converted, f"{dataset.name}/{dataset.name}.onnx")

# ONNX model info

In [26]:
from pprint import pprint
import onnxruntime

onnx_path = "Acute_Inflammations/Acute_Inflammations.onnx"
# onnx_path = "../decision_tree/decision_tree_ac.onnx"

provider = "CPUExecutionProvider"
onnx_session = onnxruntime.InferenceSession(onnx_path, providers=[provider])

print("----------------- inputs info -----------------")
input_tensors = onnx_session.get_inputs()  
for input_tensor in input_tensors:         
    
    input_info = {
        "name" : input_tensor.name,
        "type" : input_tensor.type,
        "shape": input_tensor.shape,
    }
    pprint(input_info)

print("----------------- outputs info -----------------")
output_tensors = onnx_session.get_outputs()  
for output_tensor in output_tensors:         
    
    output_info = {
        "name" : output_tensor.name,
        "type" : output_tensor.type,
        "shape": output_tensor.shape,
    }
    pprint(output_info)

----------------- inputs info -----------------
{'name': 'input', 'shape': [1, 6], 'type': 'tensor(float)'}
----------------- outputs info -----------------
{'name': 'label', 'shape': [1], 'type': 'tensor(int64)'}
{'name': 'probabilities', 'shape': [1, 4], 'type': 'tensor(float)'}
